Hablando con ChatGPT, me dice que puedo usar Google Cloud Speech-to-Text, Microsoft Azure Speech Services, IBM Watson Speech to Text.

Además, dice que puedo usar una librería de Python llamada SpeechRecognition que da una interfaz para APIs de reconocimiento de voz. Puedo usarla para integrar la API del servicio que elija.

In [1]:
!pip install --upgrade google-cloud-speech

     -------------------------------------- 271.7/271.7 kB 5.6 MB/s eta 0:00:00
     ---------------------------------------- 120.3/120.3 kB ? eta 0:00:00
     ---------------------------------------- 47.9/47.9 kB ? eta 0:00:00
     ------------------------------------- 420.6/420.6 kB 13.2 MB/s eta 0:00:00
     ---------------------------------------- 177.2/177.2 kB ? eta 0:00:00
     ------------------------------------- 223.0/223.0 kB 13.3 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 29.8 MB/s eta 0:00:00


In [5]:
from google.cloud import speech
import os

In [4]:
set GOOGLE_APPLICATION_CREDENTIALS = "C:\Users\usuario\Jarvis\jarvis-381306-4975f3ce0efb.json"

SyntaxError: invalid syntax (1002647651.py, line 1)

In [15]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/usuario/Jarvis/jarvis-381306-4975f3ce0efb.json"

In [16]:
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

C:/Users/usuario/Jarvis/jarvis-381306-4975f3ce0efb.json


In [17]:
client = speech.SpeechClient()

In [18]:
gcs_uri = "gs://cloud-samples-data/speech/brooklyn_bridge.raw"

In [19]:
audio = speech.RecognitionAudio(uri=gcs_uri)

In [20]:
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code = "en-US"
)

In [26]:
response = client.recognize(config=config, audio=audio)

#Para imprimir más de un resultado
for i in response.results:
    print(i.alternatives[0].transcript)

how old is the Brooklyn Bridge


Para hacer reconocimiento de voz de un archivo local, pero con audios cortos (máximo 60 segs):

In [61]:
def recognize_voice(audio_path):
    from google.cloud import speech
    import io
    
    client = speech.SpeechClient()
    
    with io.open(audio_path, "rb") as audio_file:
        content = audio_file.read()
        
    audio = speech.RecognitionAudio(content=content)
    
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code="es-CO"
    )
    
    response = client.recognize(config=config, audio=audio)
    
    return response

In [66]:
responses = recognize_voice("D:/USUARIO/Desktop/Grabación2.wav")

for i in responses.results:
    print(i.alternatives[0].transcript)

Hola Me llamo Lucas tengo sueño y quiero dormir


In [80]:
responses = recognize_voice("D:/USUARIO/Desktop/Grabación3.wav")

for i in responses.results:
    print(i.alternatives[0].transcript)

Este es mi madre
 y aún hay en el campo
 te amo te amo te amo te amo


In [82]:
responses = recognize_voice("D:/USUARIO/Desktop/Grabación4.wav")

for i in responses.results:
    print(i.alternatives[0].transcript)

Esta es una Esta es una prueba de sonido para ver si el modelo reconoce bien las palabras


Si quisiera hacer lo mismo pero con audios largos, entonces el código es similar, pero no se puede hacer con audios locales:
En este caso los audios deben estar en un bucket de google cloud.

In [ ]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech

    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=16000,
        language_code="en-US",
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=90)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print("Transcript: {}".format(result.alternatives[0].transcript))
        print("Confidence: {}".format(result.alternatives[0].confidence))

El reconocimiento de audio de transmisión (el que necesito) solo está disponible a través de gRCP.

In [70]:
!pip install pyaudio

     -------------------------------------- 164.1/164.1 kB 5.0 MB/s eta 0:00:00


In [71]:
from __future__ import division

import re
import sys

from google.cloud import speech

import pyaudio
from six.moves import queue

In [72]:
RATE = 16000
CHUNK = int(RATE / 10)

In [77]:
class MicrophoneStream(object):
     # Opens a recording stream as a generator yielding the audio chunks.
    
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk
        
        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True
    
    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            stream_callback=self._fill_buffer
        )
        
    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        
        self._buff.put(None)
        self._audio_interface.terminate()
        
    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        self._buff.put(in_data)
        return None, pyaudio.paContinue
    
    def generator(self):
        while not self.closed:
            chunk = self._buff.get()
            
            if chunk is None:
                return data = [chunk]
            
            while True:
                try:
                    chunk = self._buff.get